### GIL -- Global Interpreter Lock in CPython
- Python 中一个线程对应C语言中一个线程。
- GIL 使同一时刻只有一个线程在CPU上执行，无法多线程映射到多个CPU上。
- GIL 会根据字节码的行数多少决定释放CPU，IO操作时立即释放CPU。

In [2]:
# asm字节码
import dis
def add(a):
    a += 1
    return a

dis.dis(add)

  4           0 LOAD_FAST                0 (a)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_FAST               0 (a)

  5           8 LOAD_FAST                0 (a)
             10 RETURN_VALUE


In [5]:
total = 0
def add():
    global total
    for i in range(1000000):
        total += 1
        
def disc():
    global total
    for i in range(1000000):
        total -= 1
        
import threading
th1 = threading.Thread(target=add)
th2 = threading.Thread(target=disc)

th1.start()
th2.start()

th1.join()
th2.join()

total

499781

操作系统调度的最小运行单元--线程thread，能看到的是进程process（任务管理器）。

### 线程函数方式
适合小的操作。

In [22]:
import threading
import time

def getA():
    print('start get A...')
    time.sleep(2)
    print('end get A.')
    
def getB():
    print('start get B...')
    time.sleep(1)
    print('end get B.')
    
th1 = threading.Thread(target=getA)
th2 = threading.Thread(target=getB)

# th1.setDaemon(True)  # 设置守护线程，主线程不管子线程， 
# th2.setDaemon(True)

st = time.time()
th1.start()  # 产生开始线程1，主线程继续往下执行。
th2.start()

th1.join()  # 主线程等待子线程1返回，主进程被阻塞。
th2.join()  # 主线程等待子线程2

print(time.time()-st)  # 主线程等待所有子线程执行完后，结束返回

start get A...
start get B...
end get B.
end get A.
2.006112813949585


### 继承threading.Thread类的方式
适合大的操作逻辑。

In [25]:
class GetA(threading.Thread):
    def __init__(self, name):
        super().__init__(name=name)
        
    def run(self):
        print('start get A...')
        time.sleep(2)
        print('end get A.')
        
g = GetA('aaa')
st = time.time()
g.start()
g.join()
print(time.time()-st)

start get A...
end get A.
2.004106283187866


### 线程通信
- global list
- from queue import Queue 线程安全的。

In [26]:
from queue import Queue

In [29]:
q = Queue(maxsize=3)

In [39]:
q.put('a')
q.put('b')
q.put(12)

In [40]:
q.put_nowait('a')

Full: 

In [41]:
q.get()

'a'

In [42]:
q.get(block=False)

'b'

### 线程的不安全的原因 
cell3：执行+=和-=操作时(字节码)，释放了CPU（时间片），导致最后赋值时不对。

In [44]:
def add(a):
    a += 1
    
def desc(a):
    a -= 1
    
print(dis.dis(add))
print(dis.dis(desc))

  2           0 LOAD_FAST                0 (a)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_FAST               0 (a)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE
None
  5           0 LOAD_FAST                0 (a)
              2 LOAD_CONST               1 (1)
              4 INPLACE_SUBTRACT
              6 STORE_FAST               0 (a)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE
None


### 解决线程安全方法
- RLock - 能多次acquire()，但要对应相同次数release().
- Lock  - 不能两次acquire，造成死锁。

In [50]:
from threading import Lock, RLock
total = 0
lock = Lock()
def add(lock):
    global total
    
    for i in range(1000000):
        lock.acquire()
#         lock.acquire()  # 错误，Lock不能acquire()两次。
        total += 1
        lock.release()
        
        
def disc(lock):
    global total
    for i in range(1000000):
        lock.acquire()
        total -= 1
        lock.release()
        
import threading
th1 = threading.Thread(target=add, args=(lock, ))
th2 = threading.Thread(target=disc, args=(lock, ))

th1.start()
th2.start()

th1.join()
th2.join()

total

0

### 使用Lock的后果
- 造成死锁 lock.acquire()/lock.release()
- 影响程序性能，上锁解锁花时间。
- 资源竞争，程序停住，被block。

In [ ]:
# THREAD-1
def A(a, b, lock):
    lock.acquire()
#   using a do something 
#   using b do something 
    lock.release()


# THREAD-2
def A(a, b, lock):
    lock.acquire()
#   using b do something 
#   using a do something 
    lock.release()


### 用Condition实现进程间同步（信号等待）
threading.Condition()

In [56]:
import threading

class ThA(threading.Thread):
    def __init__(self, name, cond):
        super().__init__(name=name)
        self.cond = cond
        
    def run(self):
        
        with self.cond:
            self.cond.wait()
#             doing B
            print(self.name, ': 1')
            self.cond.notify()
            
            self.cond.wait()
#             doing A
            print(self.name, ': 2')
            self.cond.notify()

        
class ThB(threading.Thread):
    def __init__(self, name, cond):
        super().__init__(name=name)
        self.cond = cond
        
    def run(self):
        
        with self.cond:
            
#             doing B
            print(self.name, ': 1')
            self.cond.notify()
            self.cond.wait()
            
#             doing A
            print(self.name, ': 2')
            self.cond.notify()
            self.cond.wait()
            
cond = threading.Condition()

t1 = ThA('A', cond)
t2 = ThB('B', cond)

t1.start()
t2.start()


B : 1
A : 1
B : 2
A : 2


### Semaphore 信号量
限制线程数量。threading.Semaphore()内部维护了一个counter，用来计算lock的acquire和release。 

In [ ]:
sem = threading.Semaphore(value=3)